Vasmos a cargar el dataset de AirBnB descargado de [aquí](https://public.opendatasoft.com/explore/dataset/airbnb-listings/export/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&q=Madrid&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D&location=16,41.38377,2.15774&basemap=jawg.streets)

![](img/descargar.png)

In [ ]:
airbnb<-read.csv('airbnb-listings.csv',sep = ';')
options(repr.plot.height=4,repr.plot.width=6,repr.plot.res = 300)

Vamos a quedarnos con las columnas de mayor interés:

'City','Room.Type','Neighbourhood','Accommodates','Bathrooms','Bedrooms','Beds','Price','Square.Feet','Guests.Included','Extra.People','Review.Scores.Rating','Latitude', 'Longitude'

In [ ]:
airbnb <- subset(airbnb, 
                 select = c('City','Room.Type','Neighbourhood','Accommodates','Bathrooms','Bedrooms',
                            'Beds','Price','Square.Feet','Guests.Included',
                            'Extra.People','Review.Scores.Rating','Latitude', 'Longitude'))

Nos quedarmos solo con las entradas de Madrid para Room.Type=="Entire home/apt" y cuyo barrio (Neighbourhood) no está vacio ''
Podemos eliminar las siguientes columnas que ya no son necesarias:
"Room.Type",'City'

Llama a nuevo dataframe df_madrid.

In [ ]:
df_madrid <- airbnb[airbnb$City=="Madrid" & airbnb$Room.Type=="Entire home/apt" & airbnb$Neighbourhood != "",]
df_madrid$Square.Meters <- round(df_madrid$Square.Feet * 0.092903)
df_madrid$Room.Type <- NULL
df_madrid$City <- NULL

¿Que porcentaje de los apartamentos no muestran los metros cuadrados? Es decir, ¿cuantos tienen NA en Square.Meters?

In [ ]:
print(paste("El número total de entradas es:", nrow(df_madrid)))
print(paste("El número de entradas sin metros es:", nrow(df_madrid[is.na(df_madrid$Square.Meters),])))
print(paste("El porcentaje de entradas sin metros es: ",
            round(nrow(df_madrid[is.na(df_madrid$Square.Meters),])/nrow(df_madrid)*100, 2),"%"))

De todos los apartamentos que tienen un valor de metros cuadrados diferente de NA 
¿Que porcentaje de los apartamentos tienen 0 metros cuadrados?

In [ ]:
num_zero_sqmeters <- nrow(df_madrid[!is.na(df_madrid$Square.Meters) & df_madrid$Square.Meters == 0,])
num_with_sqmeters <- nrow(df_madrid[!is.na(df_madrid$Square.Meters),])
print(paste("El número total de apartamentos con metros cuadrados diferentes de NA es:", num_with_sqmeters))
print(paste("El número de de apartamentos con 0 metros es:", num_zero_sqmeters))
print(paste("El porcentaje de apartamentos con 0 metros es:", round(num_zero_sqmeters/num_with_sqmeters*100,2),"%"))

Reemplazar todos los 0m^2 por NA 

In [ ]:
df_madrid$Square.Meters[!is.na(df_madrid$Square.Meters) & df_madrid$Square.Meters == 0] <- NA

Son muchos, vamos a intentar crear un modelo que nos prediga cuantos son los metros cuadrados en función del resto de variables para tratar de rellenar esos NA.

Antes de eso deberíamos pintar el histograma de los metros cuadrados y ver si tenemos que filtrar algún elemento más.

In [ ]:
library(ggplot2)
ggplot(data=df_madrid, aes(x=Square.Meters))+
geom_histogram(bins=30, fill="#5ddd92", color="#446d32")+
xlab("Metros Cuadrados")

Asigna el valor NA a la columna Square.Meters de los apartamentos que tengan menos de 20 m^2

In [ ]:
# He optado por asignar NA a los menores de 25m2 porque obtenía mejor cluster y predict
df_madrid$Square.Meters[!is.na(df_madrid$Square.Meters) & df_madrid$Square.Meters < 25] <- NA

In [ ]:
library(ggplot2)
ggplot(data=df_madrid, aes(x=Square.Meters))+
geom_histogram(bins=30, fill="#5ddd92", color="#446d32")+
xlab("Metros Cuadrados")

El barrio parece ser un indicador importante para los metros cuadrados de un apartamento.

Vamos a agrupar los barrios por metros cuadrados. Podemos usar una matriz de similaridad de Tukey tal y como hicimos en el curso de estadística:

In [ ]:
tky<-TukeyHSD(aov( formula=Square.Meters~Neighbourhood, data=df_madrid ))
tky.result<-data.frame(tky$Neighbourhood)
cn <-sort(unique(df_madrid$Neighbourhood))
resm <- matrix(NA, length(cn),length(cn))
rownames(resm) <- cn
colnames(resm) <- cn
resm[lower.tri(resm) ] <- round(tky.result$p.adj,4)
resm[upper.tri(resm) ] <- t(resm)[upper.tri(resm)] 
diag(resm) <- 1
library(ggplot2)
library(reshape2)
dfResm <- melt(resm)
ggplot(dfResm, aes(x=Var1, y=Var2, fill=value))+
  geom_tile(colour = "black")+
  scale_fill_gradient(low = "white",high = "steelblue")+
  ylab("Class")+xlab("Class")+theme_bw()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1),legend.position="none")

Usando como variable de distancia: 1-resm
Dibuja un dendrograma de los diferentes barrios.


In [ ]:
library(dendextend)
options(repr.plot.height=4,repr.plot.width=12)
d <- as.dist(1 - resm)
hc <- hclust(d,method="complete")
hcd <- as.dendrogram(hc)
plot(color_branches(hcd))

Estableciendo un punto de corte en 0.9, ¿cuantos clusters aparecen?

In [ ]:
options(repr.plot.height=6,repr.plot.width=12)
plot(color_branches(hcd,h=0.9))
ncluster <- rect.hclust(hc, h=0.9)
print(paste("Número de clusters: ", length(ncluster)))

Vamos a crear una nueva columna en el dataframe df_madrid con un nuevo identificador marcado por los clusters obtenidos. Esta columna la llamaremos neighb_id

In [ ]:
cluster <- as.factor(cutree(hc,h=0.9))
df_madrid$neighb_id <- cluster[df_madrid$Neighbourhood]
str(df_madrid)

Vamos a crear dos grupos, uno test y otro train.

Tratamos de predecir los metros cuadrados en función del resto de columnas del dataframe.

In [ ]:
set.seed(1234)
idx <- sample(1:nrow(df_madrid),nrow(df_madrid)*0.7)
df_madrid_train <- df_madrid[idx,]
df_madrid_test <- df_madrid[-idx,]
paste("Número de muestras training:",nrow(df_madrid_train))
paste("Número de muestras testing:",nrow(df_madrid_test))

In [ ]:
# Probamos diferentes modelos
#Modelo_1
model_df_madrid<-lm(Square.Meters~Accommodates+Bedrooms+Bathrooms+Beds+Price+neighb_id,data=df_madrid_train)
summary(model_df_madrid)

In [ ]:
# Modelo_2. Sin Accommodates
model_df_madrid<-lm(Square.Meters~Bedrooms+Bathrooms+Beds+Price+neighb_id,data=df_madrid_train)
summary(model_df_madrid)

In [ ]:
# Modelo_3. Sin Price 
model_df_madrid<-lm(Square.Meters~Bedrooms+Bathrooms+Beds+neighb_id,data=df_madrid_train)
summary(model_df_madrid)

In [ ]:
# Modelo_4. Sin neighb_id
model_df_madrid<-lm(Square.Meters~Bedrooms+Bathrooms+Beds,data=df_madrid_train)
summary(model_df_madrid)

In [ ]:
# Finalmente nos quedamos con el modelo_3 que tiene el Adjusted R-squared más alto
model_df_madrid<-lm(Square.Meters~Bedrooms+Bathrooms+Beds+neighb_id,data=df_madrid_train)

In [ ]:
df_madrid_test$predict_result<-predict(model_df_madrid, df_madrid_test)

Mirad el histograma de los residuos sobre el conjunto de test para evaluar la calidad de vuestro modelo

In [ ]:
options(repr.plot.height=4,repr.plot.width=6,repr.plot.res = 300)
hist(df_madrid_test$Square.Meters-df_madrid_test$predict_result,10)
qqnorm(df_madrid_test$Square.Meters-df_madrid_test$predict_result)
qqline(df_madrid_test$Square.Meters-df_madrid_test$predict_result, col = 'orange', lwd =2)

In [ ]:
mean_sqerror <- sqrt(mean((df_madrid_test$Square.Meters-df_madrid_test$predict_result)^2,na.rm = TRUE))
paste("El error cuadrático medio es:",round(mean_sqerror,2))

Si tuvieramos un anuncio de un apartamento para 6 personas (Accommodates) con un precio de 80€/noche y 3 habitaciones en el barrio de Sol.
¿Cuantos metros cuadrados tendría?
¿Como varía sus metros cuadrados con cada habitación adicional?

In [ ]:
df_apartment <- data.frame(
    "Accommodates" = 6,
    "Price" = 80,
    "Bedrooms" = 3,
    "Neighbourhood" = "Sol",
    "Bathrooms" = 1,
    "Beds" = 5,
    "neighb_id" = as.factor(7)
)

In [ ]:
pr_meters_1 <- predict(model_df_madrid, df_apartment, na.action = na.pass)
paste("La predicción de metros es:", round(pr_meters_1,2))

In [ ]:
paste("Por cada habitación adicional los metros suben:",round(model_df_madrid$coefficients["Bedrooms"],2))

In [ ]:
# Comprobamos que es así aplicando el modelo a un apartamento con una habitación más
df_apartment <- data.frame(
    "Accommodates" = 6,
    "Price" = 80,
    "Bedrooms" = 4,
    "Neighbourhood" = "Sol",
    "Bathrooms" = 1,
    "Beds" = 5,
    "neighb_id" = as.factor(7)
)
pr_meters_2 <- predict(model_df_madrid, df_apartment, na.action = na.pass)
paste("La diferencia entre la primera predicción y la segunda es:",round(pr_meters_2-pr_meters_1,2))

Rellenar los Square.Meters con valor NA con el estimado con el modelo anterior.

In [ ]:
df_madrid$Predict.Meters <- predict(model_df_madrid, df_madrid)
index <- which(is.na(df_madrid$Square.Meters))
df_madrid$Square.Meters[index] <- df_madrid$Predict.Meters[index]
df_madrid$Square.Meters[df_madrid$Square.Meters<25] <- NA
head(df_madrid)

Usar PCA para encontrar el apartamento más cercano a uno dado.

Este algoritmo nos ayudaría a dado un apartamento que el algoritmo nos devolvería los 5 apartamentos más similares.


Crearemos una función tal que le pasemos un apartamento con los siguientes datos:
* Accommodates	
* Bathrooms	
* Bedrooms	
* Beds	
* Price	
* Guests.Included	
* Extra.People	
* Review.Scores.Rating	
* Latitude	
* Longitude	
* Square.Meters

y nos devuelva los 5 más similares de:

In [ ]:
# Creamos un sub dataframe con las columnas numéricas y quitamos las filas con NA
df_sub_madrid <- subset(df_madrid, 
                 select = c('Accommodates','Bathrooms','Bedrooms','Beds','Price','Guests.Included',
                            'Extra.People','Review.Scores.Rating','Latitude', 'Longitude', 'Square.Meters'))
df_sub_madrid <- na.omit(df_sub_madrid)

# Creamos el apartamento del que queremos buscar los mas parecidos, cogiémdolo del propio dataframe
df_apartment <- df_sub_madrid[78,c('Accommodates','Bathrooms','Bedrooms','Beds','Price','Guests.Included',
                            'Extra.People','Review.Scores.Rating','Latitude', 'Longitude', 'Square.Meters')]
df_apartment

In [ ]:
# Función que devuelve los 5 apartamentos más parecidos
# Le pasamos el dataframe limpio de NA, el item que queremos comparar
most_similar_apts <- function(df_complete, new_item, num_items=5){
    pr_apt <- prcomp(df_complete,center = TRUE, scale. = TRUE)
    t_apt <- predict(pr_apt, newdata = new_item)
    dist<-rowSums((t_apt[rep(1, times = nrow(pr_apt$x)), ]-pr_apt$x)^2)
    df_complete[order(dist)[1:num_items],]
}

In [ ]:
most_similar_apts(df_sub_madrid, df_apartment)

**Podemos observar como la primera aparición es el propio apartamento que estaba en el dataframe y luego las cuatro mas similares.**

Bonus: ¿de todas las coordenadas PCA cual es la que mejor permite clasificar por id de barrio?

¿Por qué?

In [ ]:
pr_apt <- prcomp(df_sub_madrid,center = TRUE, scale. = TRUE)

In [ ]:
pr_apt$x